In [1]:
from premise import *
from datapackage import Package
import bw2data, bw2io, bw2calc
from unfold import Unfold
bw2data.projects.set_current("new")

In [2]:
clear_cache()

Cache folder cleared!


In [3]:
combinations = [
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 5, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00}
]

In [4]:
scenarios = [
    {"model": "remind", "pathway":"SSP2-Base", "year": 2030},
    {"model": "remind", "pathway":"SSP5-Base", "year": 2080},
]

for model in [
    "cutoff",
    "consequential"
]:

    for ei in [
        "3.6",
        "3.7",
        #"3.7.1",
        "3.8",
        #"3.9",
        "3.9.1"

    ]:
        
        if ei not in ["3.9", "3.9.1"]:
            bw2data.projects.set_current("new4")
        else:
            bw2data.projects.set_current("ei39")

        for source in [
            "brightway",
            #"ecospold"
        ]:
            for export in [
                "brightway",
                #"superstructure",
                #"simapro",
                #"matrices",
                #"datapackage"
            ]:

                ndb = NewDatabase(
                        scenarios = scenarios,
                        source_db= f"ecoinvent {ei} {model}" if source == "brightway" else None,
                        source_type=source,
                        source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
                        source_version=ei,
                        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
                        system_model=model,
                        system_args=combinations[0],
                        additional_inventories= [ # <-- this is NEW
                            {"filepath": "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-PV.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
                    ] # <-- this is NEW
                )
                

                print("#############################################################################################################")
                print(ei, source, export)

                ndb.update_all()
                #ndb.update_electricity()
                #ndb.update_buses()
                #ndb.update_cars()
                #ndb.update_trucks()
                #ndb.update_two_wheelers()


                if export == "brightway":
                    ndb.write_db_to_brightway([
                        "test1",
                        "test2"
                        ]
                    )
                    lca = bw2calc.LCA({bw2data.Database("test1").random(): 1})
                    lca.lci()
                    lca = bw2calc.LCA({bw2data.Database("test2").random(): 1})
                    lca.lci()
                elif export == "superstructure":
                    ndb.write_superstructure_db_to_brightway("test_super")
                elif export == "simapro":
                    ndb.write_db_to_simapro()
                elif export == "matrices":
                    ndb.write_db_to_matrices()
                else:
                    ndb.write_datapackage(name="test")
                    u = Unfold("export/datapackage/test.zip")
                    u.unfold(dependencies={"ecoinvent": f"ecoinvent {ei} {model}", "biosphere3": "biosphere3"}, superstructure=True)
                
            
            


premise v.(1, 7, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18121/18121 [00:00<00:00, 108432.32it/s]


Adding exchange data to activities


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 615644/615644 [00:29<00:00, 20700.10it/s]


Filling out exchange data


...

Filling out exchange data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:03<00:00, 6042.41it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.13 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datase

Process SpawnPoolWorker-51:
Process SpawnPoolWorker-52:
Process SpawnPoolWorker-54:
Process SpawnPoolWorker-56:
Process SpawnPoolWorker-50:
Process SpawnPoolWorker-55:
Process SpawnPoolWorker-49:
Process SpawnPoolWorker-53:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.10/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/Users/romain/GitHub/premise/premise/electricity.py", line 197, in _update_electricity


KeyboardInterrupt: 

In [1]:
import unfold
print(unfold.__version__)
import bw2data
from unfold import Unfold

(1, 1, 2)


In [2]:
bw2data.projects.set_current("new4")

In [3]:
u = Unfold("export/datapackage/test.zip")
#u.unfold(dependencies={"ecoinvent": f"ecoinvent 3.8 cutoff", "biosphere3": "biosphere3"})

The data package contains the following scenarios:
+-----+---------------------------+------------------------------------------------------------------------+
| No. |          Scenario         |                              Description                               |
+-----+---------------------------+------------------------------------------------------------------------+
|  0  | remind - SSP2-Base - 2030 | Prospective db, based on REMIND, pathway SSP2-BASE, for the year 2030. |
|  1  | remind - SSP5-Base - 2080 | Prospective db, based on REMIND, pathway SSP5-BASE, for the year 2080. |
+-----+---------------------------+------------------------------------------------------------------------+

To unfold all scenarios, `unfold()`.
To unfold a specific scenario, `unfold(scenarios=[1,])`.


In [4]:
u.unfold(superstructure=True, export_dir="/Users/romain/Documents")

The following databases are necessary to unfold the scenarios:
+-----+------------+--------------+---------+
| No. | Dependency | System model | Version |
+-----+------------+--------------+---------+
|  1  | ecoinvent  |   cut-off    |   3.7   |
|  2  | biosphere3 |              |         |
+-----+------------+--------------+---------+

The following databases are available in your project:
+-----+-------------------------------------------------+
| No. |                     Database                    |
+-----+-------------------------------------------------+
|  1  |                    biosphere3                   |
|  2  |               ecoinvent 3.8 cutoff              |
|  3  |               ecoinvent 3.7 cutoff              |
|  4  |               ecoinvent 3.6 cutoff              |
|  5  |          ecoinvent_remind_SSP2-NPi_2005         |
|  6  |          ecoinvent_remind_SSP2-NPi_2010         |
|  7  |                    test_super                   |
|  8  |                  

Indicate the database number for dependency 1:  3
Indicate the database number for dependency 2:  1


Extracting additional inventories...
Writing scenario difference file...
Generating superstructure database...
Scenario difference file exported to test.csv!

Writing superstructure database...
Database test already exists: it will be overwritten.


Writing activities to SQLite3 database:


Vacuuming database 


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Title: Writing activities to SQLite3 database:
  Started: 09/27/2023 17:51:16
  Finished: 09/27/2023 17:51:38
  Total time elapsed: 00:00:22
  CPU %: 95.10
  Memory %: 8.57
Created database: test


In [5]:
u.scenario_df.head()

,from activity name,from reference product,from location,from categories,from database,from key,from unit,to activity name,to reference product,to location,to categories,to unit,to database,to key,flow type,image - SSP2-Base - 2030,remind - SSP5-Base - 2080,flow id
0,"market group for electricity, low voltage","electricity, low voltage",EUR,None,test,"(test, 5bc91358e7b74cc1addcda3a21da61bd)",kilowatt hour,"market for electricity, low voltage","electricity, low voltage",LU,None,kilowatt hour,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",technosphere,0.0,1.0,"(market for electricity, low voltage, electric..."
1,"market for sulfur hexafluoride, liquid","sulfur hexafluoride, liquid",RER,None,test,"(test, 5716dfc0ef690eac57ce6c20a0de4fc0)",kilogram,"market for electricity, low voltage","electricity, low voltage",LU,None,kilowatt hour,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",technosphere,0.0,0.0,"(market for electricity, low voltage, electric..."
2,electricity voltage transformation from medium...,"electricity, low voltage",LU,None,test,"(test, 7654225c9b8bf414f28d91aad81cfa22)",kilowatt hour,"market for electricity, low voltage","electricity, low voltage",LU,None,kilowatt hour,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",technosphere,0.0,0.0,"(market for electricity, low voltage, electric..."
3,"market for electricity, low voltage","electricity, low voltage",LU,None,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",kilowatt hour,"market for electricity, low voltage","electricity, low voltage",LU,None,kilowatt hour,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",technosphere,0.0,0.0,"(market for electricity, low voltage, electric..."
4,"electricity production, photovoltaic, 3kWp sla...","electricity, low voltage",LU,None,test,"(test, 596638ecc3b266b3fa04a0ae24231cbd)",kilowatt hour,"market for electricity, low voltage","electricity, low voltage",LU,None,kilowatt hour,test,"(test, c0d7a551aba40d44b9447661b6811a6f)",technosphere,0.0,0.0,"(market for electricity, low voltage, electric..."


In [24]:
for ds in db:
    if ds["name"].startswith("ethanol production, via fermentation, from poplar, system expansion"):
        print(ds)

In [17]:
for ds in u.database:
    if ds["name"].startswith("ethanol production, via fermentation, from poplar, system expansion"):
        for e in ds["exchanges"]:
            if e["type"] == "production":
                print(e)

{'name': 'ethanol production, via fermentation, from poplar, system expansion', 'amount': 1.0, 'unit': 'kilogram', 'location': 'US', 'type': 'production', 'product': 'ethanol, from poplar'}


In [9]:
for ds in ndb.scenarios[1]["database"]:
    for e in ds["exchanges"]:
        if e["amount"] == 1.0057:
            print(ds["name"], ds["location"], e["name"]) 

In [1]:
from premise import *
from datapackage import Package
import brightway2 as bw
from unfold import Unfold
bw.projects.set_current("ei39")

In [2]:
clear_cache()

Cache folder cleared!


In [3]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
    {"model": "remind", "pathway":"SSP5-Base", "year": 2080},
]



ndb = NewDatabase(
        scenarios = scenarios,
        source_db= "ecoinvent EN15804 3.9.1",
        source_type="brightway",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        system_model="cutoff",
        additional_inventories= [ # <-- this is NEW
            {"filepath": "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-PV.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
    ] # <-- this is NEW
)
ndb.update_all()
ndb.write_db_to_brightway([
    "test1",
    "test2"
    ]
)
lca = bw.LCA({bw.Database("test1").random(): 1})
lca.lci()
lca = bw.LCA({bw.Database("test2").random(): 1})
lca.lci()


premise v.(1, 5, 0, 'beta', 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+--

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21277/21277 [00:00<00:00, 302188.16it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 679195/679195 [00:56<00:00, 12027.78it/s]


Filling out exchange data


...

Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.20 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 worksheets in 0.01 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: mig

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Title: Writing activities to SQLite3 database:
  Started: 04/23/2023 16:27:33
  Finished: 04/23/2023 16:28:05
  Total time elapsed: 00:00:32
  CPU %: 97.80
  Memory %: 36.63
Created database: test1
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Database test2 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Title: Writing activities to SQLite3 database:
  Started: 04/23/2023 16:36:39
  Finished: 04/23/2023 16:37:07
  Total time elapsed: 00:00:27
  CPU %: 98.00
  Memory %: 36.73
Created database: test2
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [4]:
ndb.scenarios[1]["iam data"].other_vars.sel(variables="Temperature|Global Mean", region="World")

<xarray.DataArray 'value' (year: 19)>
array([0.882371, 0.894388, 0.993749, 1.114845, 1.239434, 1.35875 ,
       1.502813, 1.679966, 1.908864, 2.186055, 2.440907, 2.69551 ,
       3.200126, 3.682789, 4.123068, 4.513573, 4.860211, 5.423776,
       5.879446])
Coordinates:
    region     <U5 'World'
    variables  <U23 'Temperature|Global Mean'
  * year       (year) int64 2005 2010 2015 2020 2025 ... 2100 2110 2130 2150

In [5]:
for ds in bw.Database("ecoinvent 3.8 cutoff"):
    for e in ds.exchanges():
        if e["type"] == "biosphere" and any(i in e["name"] for i in ["HFC", "HCFC", "CFC", "Halon"]):
            print(ds["name"], e["name"])

uranium production, centrifuge, enriched 4.2% Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
transport, trolleybus Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
transport, freight, lorry with refrigeration machine, 3.5-7.5 ton, EURO3, R134a refrigerant, cooling Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
heat and power co-generation, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
transport, freight, lorry with refrigeration machine, 3.5-7.5 ton, EURO4, R134a refrigerant, cooling Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
heat and power co-generation, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
uranium production, diffusion, enriched 4.0% Ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, CFC-114
electricity production, hard coal Ethane, 1,1,1-tric

In [1]:
from premise.geomap import Geomap
geo = Geomap("remind")

In [ ]:
geo.geo.add_definitions

In [5]:
for ds in ndb.database:
    if ds["name"] == "market for heat, from steam, in chemical industry":
        print(ds["name"], ds["reference product"], ds["location"], ds["unit"])

market for heat, from steam, in chemical industry heat, from steam, in chemical industry RER megajoule
market for heat, from steam, in chemical industry heat, from steam, in chemical industry RoW megajoule


In [6]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["input"][0] == '4b344fb0e0cd348af7cf9007812638b0':
            print(e)

KeyError: 'input'

In [4]:
from premise.activity_maps import InventorySet

i = InventorySet(
    ndb.database,
)

In [7]:
i.generate_fuel_map()

{'hard coal': {'market for hard coal', 'market for hard coal, run-of-mine'},
 'lignite': {'market for lignite'},
 'petroleum coke': {'market for petroleum coke'},
 'wood pellet': {'market for wood pellet, measured as dry mass'},
 'wood chips': {'market for wood chips and particles, willow',
  'market for wood chips, dry, measured as dry mass',
  'market for wood chips, from post-consumer wood, measured as dry mass',
  'market for wood chips, wet, measured as dry mass'},
 'natural gas': {'natural gas production',
  'natural gas production, liquefied',
  'natural gas production, unprocessed, at extraction',
  'natural gas, at production',
  'natural gas, at production offshore',
  'natural gas, at production onshore'},
 'light fuel oil': {'light fuel oil production, from waste polyethylene',
  'light fuel oil production, petroleum refinery operation',
  'light fuel oil, import from Europe',
  'light fuel oil, import from RoW'},
 'biogas': {'Biogas, burned in power plant/NGCC, no CCS',
  